<a href="https://colab.research.google.com/github/DrAlexSanz/nlpv2-course/blob/master/Markov_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text generation

Markov models are typical first order models (word depends only on previous word). Let's use a second order model. Words depend on the previous TWO words.



In [30]:
import numpy as np
import string
np.random.seed(1234)

In [31]:
initial = {} # Start of a phrase, initial word distribution
first_order = {} # Second word only
second_order = {} # second order transition probs

In [32]:
def remove_punkt(s):
    return s.translate(str.maketrans("", "", string.punctuation))

In [37]:
def add2dict(dictionary, key, value):
    """
    keys are tuples. The previous two words. Values are the probabilities.
    """
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

    return


In [38]:
for line in open("robert_frost.txt"):
    tokens = remove_punkt(line.rstrip().lower()).split()
    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            # Distribution of first word
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i-1]
            if i == T-1: # The one before the last word --> probability of ending hte line
                add2dict(second_order, (t_1, t), "END")
            
            # Not elif, two independent checks
            if i == 1: # First order, distribution of second word given the first word
                add2dict(first_order, t_1, t)

            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [40]:
# Normalize distributions

initial_total = sum(initial.values())

for t, c in initial.items():
    initial[t] = c/initial_total

In [41]:
# convert [cat, cat, cat, dog, dog, dog, mouse...]
# to {cat: 0.5, dog: 0.4, mouse: 0.1}

def list2pdicts(tokens_list):
    d = {}
    n = len(tokens_list)
    
    for t in tokens_list: # loop through each token. Each time I see it, I count it.
        d[t] = d.get(t, 0.) + 1
    
    for t, c in d.items(): # Normalize the count from before
        d[t] = c/n

    return d


In [42]:
for t_1, token_list in first_order.items():
    first_order[t_1] = list2pdicts(token_list)

In [46]:
for k, token_list in second_order.items():
    second_order[k] = list2pdicts(token_list) # And this will be a dictionary of dictionaries

In [51]:
def sample_word(dictionary):
    p_0 = np.random.random()
    cumulative = 0
    for t, p in dictionary.items():
        cumulative += p

        if p_0 < cumulative:
            return t
    assert False # Shouldn't reach here

In [57]:
def generate_n_lines(n):
    """ Generate n sentences, 1 sentence at a time"""
    for i in range(n):
        sentence = []
        # Initial word
        w_0 = sample_word(initial)
        sentence.append(w_0)

        # Second word
        w_1 = sample_word(first_order[w_0])
        sentence.append(w_1)

        # Second order transitions until END

        while True:
            w_2 = sample_word(second_order[(w_0, w_1)])
            if w_2 == "END":
                break
            sentence.append(w_2)
            w_0 = w_1
            w_1 = w_2
        print(" ".join(sentence))

    return


In [61]:
generate_n_lines(12)

he gained no foothold but pursued
does the rain seem to say
me im not blaming him
with our arms at the level of our united strengths to do right
heres hoping she gets her drink and be one traveler long i lay
and when ive said why shouldnt they be married
with all this now too much to keep accounts
dont ask for better
i guess estelle and i can
should say our stock was petered out
hes been in a window
too lofty and original to rage


In [66]:
len(initial) ** 3

dict_1_elem = len(initial)
dict_2_elem = 0
for elem in first_order.items():
    dict_2_elem += len(elem)
dict_2_elem
dict_3_elem = 0
for elem in second_order.items():
    for elem2 in elem:
        dict_3_elem += len(elem2)
dict_3_elem

22920